In [1]:
import numpy as np
import pandas as pd
import psycopg2
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
import statsmodels.api as sm
import configparser
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Activation
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import History
import warnings
warnings.filterwarnings('ignore')

In [2]:
def fetch_data():
    config = configparser.ConfigParser()
    config.read('db_config.ini')

    host = config['database']['host']
    port = config['database'].getint('port')
    user = config['database']['user']
    password = config['database']['password']
    database = config['database']['database']

    connection = psycopg2.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=database
    )

    df1 = pd.read_sql('SELECT * FROM turbofan_engine_data', con=connection)
    df2 = pd.read_sql('SELECT * FROM turbofan_rul_data', con=connection)

    connection.close()

    return df1, df2

In [3]:
df1, df2 = fetch_data()

In [4]:
def clean_data(df):
    # Handling missing values
    na_counts_before = df.isna().sum()
    if na_counts_before.sum() > 0:
        print(f"Rows with missing values before: {na_counts_before.sum()}")
        df = df.dropna(axis=0)
        na_counts_after = df.isna().sum()
        print(f"Rows with missing values after: {na_counts_after.sum()}")
    else:
        print("No missing values found. Proceeding with the original DataFrame.")

    # Handling duplicates
    duplicates_before = df[df.duplicated(keep='first')]
    if len(duplicates_before) > 0:
        print(f"Duplicates before: {len(duplicates_before)}")
        df = df.drop_duplicates(keep='first')
        duplicates_after = df[df.duplicated(keep='first')]
        print(f"Duplicates after: {len(duplicates_after)}")
    else:
        print("No duplicate rows found. Proceeding with the original DataFrame.")

    return df

In [5]:
df1 = clean_data(df1)
df2 = clean_data(df2)

No missing values found. Proceeding with the original DataFrame.
No duplicate rows found. Proceeding with the original DataFrame.
No missing values found. Proceeding with the original DataFrame.
No duplicate rows found. Proceeding with the original DataFrame.


In [6]:
train_df = df1[df1["source"] == 0].copy()
test_df = df1[df1["source"] == 1].copy()

# **Deriving RUL Column**

In [7]:
def add_rul_train(df):
    max_cycles_train = df.groupby('engine')['cycle'].max().reset_index()
    max_cycles_train.columns = ['engine', 'max_cycle_train']
    df = df.merge(max_cycles_train, on='engine', how='left')
    df['RUL'] = df['max_cycle_train'] - df['cycle']
    df.drop(['max_cycle_train'], axis=1, inplace=True)
    return df

In [8]:
train_df_with_rul = add_rul_train(train_df)

In [9]:
df_train = train_df_with_rul.copy()
df_test = test_df
y_true = df2.copy()

In [10]:
df_train.shape, df_test.shape, y_true.shape

((53759, 29), (33991, 28), (259, 2))

In [11]:
features = df_train.columns.drop(['index','source','engine', 'cycle', 'RUL'])

In [12]:
sc = MinMaxScaler(feature_range=(-1,1))

df_train[features] = sc.fit_transform(df_train[features])
df_test[features] = sc.transform(df_test[features])

# **Preparing Data**

In [13]:
#function to prepare train data into (samples, time steps, features)

def gen_train(id_df, seq_length, seq_cols):

    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    lstm_array=[]

    for start, stop in zip(range(0, num_elements-seq_length+1), range(seq_length, num_elements+1)):
        lstm_array.append(data_array[start:stop, :])

    return np.array(lstm_array)

In [14]:
def gen_target(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length-1:num_elements+1]

In [15]:
# Function to prepare test data into (samples, time steps, features).

def gen_test(id_df, seq_length, seq_cols, mask_value):

    df_mask = pd.DataFrame(np.zeros((seq_length-1, id_df.shape[1])), columns=id_df.columns)
    df_mask[:] = mask_value

    id_df = pd.concat([df_mask, id_df], ignore_index=True)

    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    lstm_array = []

    start = num_elements - seq_length
    stop = num_elements

    lstm_array.append(data_array[start:stop, :])

    return np.array(lstm_array)

In [16]:
sequence_length = 50
mask_value = 0

In [17]:
x_train=np.concatenate(list(list(gen_train(df_train[df_train['engine']==unit], sequence_length, features)) for unit in df_train['engine'].unique()))
print(x_train.shape)

(41019, 50, 24)


In [18]:
y_train = np.concatenate(list(list(gen_target(df_train[df_train['engine']==unit], sequence_length, "RUL")) for unit in df_train['engine'].unique()))
print(y_train.shape)

(41019,)


In [19]:
x_test=np.concatenate(list(list(gen_test(df_test[df_test['engine']==unit], sequence_length, features, mask_value)) for unit in df_test['engine'].unique()))
print(x_test.shape)

(259, 50, 24)


In [20]:
y_test = y_true.RUL.values
print(y_test.shape)

(259,)


In [21]:
nb_features = x_train.shape[2]
nb_out = 1

In [22]:
nb_features

24

# **Model Training**

In [23]:
history = History()

model = Sequential()
model.add(LSTM(
         units=100,
         return_sequences=True,
         input_shape=(sequence_length, nb_features)))
model.add(Dropout(0.2))
model.add(LSTM(
          units=100,
          return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(
          units=100,
          return_sequences=False))
model.add(Dense(units=50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='relu'))
model.add(Activation("relu"))

model.compile(loss="mse", optimizer="rmsprop", metrics=['mse'])

In [24]:
model.fit(x_train, y_train, epochs=100, batch_size=128, validation_split=0.1, verbose=1, callbacks=[history])

Epoch 1/100
289/289 [==============================] - 124s 403ms/step - loss: 4142.7979 - mse: 4142.7979 - val_loss: 4206.7656 - val_mse: 4206.7656
Epoch 2/100
289/289 [==============================] - 116s 401ms/step - loss: 2295.8374 - mse: 2295.8374 - val_loss: 1850.7705 - val_mse: 1850.7705
Epoch 3/100
289/289 [==============================] - 113s 391ms/step - loss: 1621.9384 - mse: 1621.9384 - val_loss: 1970.2701 - val_mse: 1970.2701
Epoch 4/100
289/289 [==============================] - 119s 411ms/step - loss: 1453.2338 - mse: 1453.2338 - val_loss: 1775.2072 - val_mse: 1775.2072
Epoch 5/100
289/289 [==============================] - 117s 404ms/step - loss: 1319.7524 - mse: 1319.7524 - val_loss: 1492.0029 - val_mse: 1492.0029
Epoch 6/100
289/289 [==============================] - 114s 394ms/step - loss: 1246.2896 - mse: 1246.2896 - val_loss: 2209.3464 - val_mse: 2209.3464
Epoch 7/100
289/289 [==============================] - 115s 397ms/step - loss: 1195.9802 - mse: 1195.9802 

In [25]:
def evaluate_model(y_true, y_pred):

    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f'RMSE: {rmse:.0f}')

    mae = mean_absolute_error(y_true, y_pred)
    print(f'MAE: {mae:.0f}')

    r2 = r2_score(y_true, y_pred)
    print(f'R² Score: {r2:.4f}')

In [26]:
y_train_pred = model.predict(x_train).flatten()

print("Training Set Performance:")
evaluate_model(y_train, y_train_pred)

1282/1282 [==============================] - 59s 45ms/step
Training Set Performance:
RMSE: 16
MAE: 7
R² Score: 0.9218


In [27]:
y_test_pred = model.predict(x_test).flatten()

print("\nTest Set Performance:")
evaluate_model(y_test, y_test_pred)

9/9 [==============================] - 0s 41ms/step

Test Set Performance:
RMSE: 42
MAE: 30
R² Score: 0.3864
